In [1]:
import sys;
sys.path.insert(0, '..')

## Chapter 6 Code Snippets and Listings

### Finding good outcomes with oracles (section 6.1.1)

Assume we want to find one special item in a list of $N = 8$ items, but we do not know where the item is. We can represent each of the items as an outcome of a quantum state with $n = 3$ qubits. Any randomly selected outcome has the same likelihood of being the good outcome. We represent this with equal amplitudes for each outcome:

In [2]:
from math import sqrt

n = 3
state = [1/sqrt(2**n) for _ in range(2**n)]

In [3]:
from util import print_state_table

print_state_table(state)


Outcome  Binary  Amplitude           Magnitude  Direction  Amplitude Bar             Probability
------------------------------------------------------------------------------------------------
0        000     0.3536 + i0.0000    0.3536        0.00°   ████████                  0.125 
1        001     0.3536 + i0.0000    0.3536        0.00°   ████████                  0.125 
2        010     0.3536 + i0.0000    0.3536        0.00°   ████████                  0.125 
3        011     0.3536 + i0.0000    0.3536        0.00°   ████████                  0.125 
4        100     0.3536 + i0.0000    0.3536        0.00°   ████████                  0.125 
5        101     0.3536 + i0.0000    0.3536        0.00°   ████████                  0.125 
6        110     0.3536 + i0.0000    0.3536        0.00°   ████████                  0.125 
7        111     0.3536 + i0.0000    0.3536        0.00°   ████████                  0.125 



Let's look at an example of the classical implementation of a phase oracle from chapter 8. The function `predicate` identifies good outcome 3 as the only good outcome.

In [4]:
predicate = lambda k: True if k == 3 else False

We can use this predicate for the classical implementation of a phase oracle:

In [5]:
def oracle(state, predicate):
    for item in range(len(state)):
        if predicate(item):
            state[item] *= -1
            
oracle(state, predicate)

In [6]:
print_state_table(state)


Outcome  Binary  Amplitude           Magnitude  Direction  Amplitude Bar             Probability
------------------------------------------------------------------------------------------------
0        000     0.3536 + i0.0000    0.3536        0.00°   ████████                  0.125 
1        001     0.3536 + i0.0000    0.3536        0.00°   ████████                  0.125 
2        010     0.3536 + i0.0000    0.3536        0.00°   ████████                  0.125 
3        011    -0.3536 + i0.0000    0.3536      180.00°   ████████                  0.125 
4        100     0.3536 + i0.0000    0.3536        0.00°   ████████                  0.125 
5        101     0.3536 + i0.0000    0.3536        0.00°   ████████                  0.125 
6        110     0.3536 + i0.0000    0.3536        0.00°   ████████                  0.125 
7        111     0.3536 + i0.0000    0.3536        0.00°   ████████                  0.125 



### Computing similarity with inner products (section 6.1.2)

For example, say we bought 4 apples, 2 oranges, 2 peaches, and 3 bananas. Let's assume one apple costs \\$1.20, one orange costs \\$1.50, one peach costs \\$2.00, and one banana costs $0.70. We can express these quantities and prices as lists:

In [7]:
quantities = [4, 2, 2, 3]
prices = [1.2, 1.5, 2, 0.7]

To calculate the total price for the list of items, we multiply the quantity of each item by its price and add the results together:

In [8]:
print(sum([quantities[k] * prices[k] for k in range(len(quantities))]))

13.9


The following Python code computes the inner product of two state vectors, `state1` and `state2`:

`sum(state1[k]*state2[k].conjugate() for k in range(len(state1)))`

Listing 6.1 Computing the inner product of two state vectors

In [9]:
def inner(v1, v2):
    assert(len(v1) == len(v2))
    return sum(z1*z2.conjugate() for z1, z2 in zip(v1, v2))

### The inversion operator (6.1.3)

Listing 6.2 Classical implementation of the inversion operator

In [10]:
def inversion(original, current):
    proj = inner(original, current)
    for k in range(len(current)):
        current[k] = 2*proj*original[k] - current[k]

Previous example three-qubit state obtained by applying an oracle that tags the outcome 3 to a state where the amplitudes are in equal superposition:

In [11]:
n = 3
state = [1/sqrt(2**n) for _ in range(2**n)]
s = state.copy() # copies the original state to use for the inversion

oracle(state, predicate)

Now, we can apply the inversion operator that reflects the current state into the original state (the state before the oracle was applied):

In [12]:
inversion(s, state) # the parameter s is the initial state before the oracle was applied

In [13]:
print_state_table(state)


Outcome  Binary  Amplitude           Magnitude  Direction  Amplitude Bar             Probability
------------------------------------------------------------------------------------------------
0        000     0.1768 + i0.0000    0.1768        0.00°   ████                      0.0312
1        001     0.1768 + i0.0000    0.1768        0.00°   ████                      0.0312
2        010     0.1768 + i0.0000    0.1768        0.00°   ████                      0.0312
3        011     0.8839 + i0.0000    0.8839        0.00°   █████████████████████     0.7812
4        100     0.1768 + i0.0000    0.1768        0.00°   ████                      0.0312
5        101     0.1768 + i0.0000    0.1768        0.00°   ████                      0.0312
6        110     0.1768 + i0.0000    0.1768        0.00°   ████                      0.0312
7        111     0.1768 + i0.0000    0.1768        0.00°   ████                      0.0312



Let's look at another example, with a random $n = 3$ qubit state and good outcome 5:

In [14]:
from util import generate_state

n = 3
state = generate_state(n)

In [15]:
print_state_table(state)


Outcome  Binary  Amplitude           Magnitude  Direction  Amplitude Bar             Probability
------------------------------------------------------------------------------------------------
0        000    -0.1850 - i0.1342    0.2285     -144.96°   █████                     0.0522
1        001     0.2070 + i0.0254    0.2086        7.00°   █████                     0.0435
2        010     0.2726 + i0.2780    0.3894       45.56°   █████████                 0.1516
3        011     0.4867 + i0.1080    0.4985       12.51°   ███████████               0.2485
4        100    -0.2028 + i0.0177    0.2036      175.10°   ████                      0.0414
5        101     0.3219 + i0.0871    0.3335       15.14°   ████████                  0.1112
6        110    -0.1712 - i0.3447    0.3849     -116.59°   █████████                 0.1481
7        111     0.3190 + i0.3188    0.4509       44.98°   ██████████                0.2034



In [16]:
s = state.copy()

predicate = lambda k: True if k == 5 else False
oracle(state, predicate)

In [17]:
print_state_table(state)


Outcome  Binary  Amplitude           Magnitude  Direction  Amplitude Bar             Probability
------------------------------------------------------------------------------------------------
0        000    -0.1850 - i0.1342    0.2285     -144.96°   █████                     0.0522
1        001     0.2070 + i0.0254    0.2086        7.00°   █████                     0.0435
2        010     0.2726 + i0.2780    0.3894       45.56°   █████████                 0.1516
3        011     0.4867 + i0.1080    0.4985       12.51°   ███████████               0.2485
4        100    -0.2028 + i0.0177    0.2036      175.10°   ████                      0.0414
5        101    -0.3219 - i0.0871    0.3335     -164.14°   ████████                  0.1112
6        110    -0.1712 - i0.3447    0.3849     -116.59°   █████████                 0.1481
7        111     0.3190 + i0.3188    0.4509       44.98°   ██████████                0.2034



Next, we perform the inversion:

In [18]:
inversion(s, state)

In [19]:
print_state_table(state)


Outcome  Binary  Amplitude           Magnitude  Direction  Amplitude Bar             Probability
------------------------------------------------------------------------------------------------
0        000    -0.1027 - i0.0745    0.1269     -144.96°   ███                       0.0161
1        001     0.1149 + i0.0141    0.1158        7.00°   ██                        0.0134
2        010     0.1513 + i0.1543    0.2161       45.56°   █████                     0.0467
3        011     0.2701 + i0.0599    0.2767       12.50°   ██████                    0.0766
4        100    -0.1126 + i0.0098    0.113       175.30°   ██                        0.0128
5        101     0.8226 + i0.2227    0.8522       15.15°   ████████████████████      0.7262
6        110    -0.0950 - i0.1913    0.2136     -116.59°   █████                     0.0456
7        111     0.1770 + i0.1769    0.2503       44.98°   ██████                    0.0626



**Visualizing inversion by the mean**

Let's create a $n = 3$ qubit state in equal superposition and apply an oracle for good outcome 3:

In [20]:
n = 3
state = [1/sqrt(2**n) for _ in range(2**n)]

s = state.copy()

predicate = lambda k: True if k == 3 else False
oracle(state, predicate)

We can check that the mean of the amplitudes is equal to the quantity defined above for bad outcomes $k$:

In [21]:
from util import is_close

amplitude_mean = sum(state)/2**n

proj = inner(s, state)
for k in range(len(state)):
    if k != 3:
        assert is_close(proj*state[k], amplitude_mean)

Now, we can simulate the inversion by the mean with the following Python code:

In [22]:
for k in range(len(state)):
    state[k] = 2*amplitude_mean-state[k]

In [23]:
print_state_table(state)


Outcome  Binary  Amplitude           Magnitude  Direction  Amplitude Bar             Probability
------------------------------------------------------------------------------------------------
0        000     0.1768 + i0.0000    0.1768        0.00°   ████                      0.0312
1        001     0.1768 + i0.0000    0.1768        0.00°   ████                      0.0312
2        010     0.1768 + i0.0000    0.1768        0.00°   ████                      0.0312
3        011     0.8839 + i0.0000    0.8839        0.00°   █████████████████████     0.7812
4        100     0.1768 + i0.0000    0.1768        0.00°   ████                      0.0312
5        101     0.1768 + i0.0000    0.1768        0.00°   ████                      0.0312
6        110     0.1768 + i0.0000    0.1768        0.00°   ████                      0.0312
7        111     0.1768 + i0.0000    0.1768        0.00°   ████                      0.0312



### Putting it together: the Grover iterate (section 6.1.4)

Listing 6.3 Classical implementation of the Grover iterate

In [24]:
from math import cos

def classical_grover(state, predicate, iterations):
    s = state.copy()
    items = [k for k in range(len(state)) if predicate(k)]

    p = sum([abs(s[k])**2 for k in items]) # uses the probability of measuring a good outcome to define an angle theta
    theta = asin(sqrt(p))
    assert is_close(inner(s, state), 1)

    for it in range(1, iterations + 1):
        oracle(state, predicate)
        inversion(s, state)
        assert is_close(inner(s, state), cos(2*it*theta)) # the inner product after operator A is applied and the state after j Grover iterations is cos(2j theta)

        p = sum([abs(state[k])**2 for k in items]) # finds the new probability of measuring a good outcome
        assert is_close(p, sin((2*it + 1)*theta)**2) # checks that the probability of good outcomes is sin^2((2j+1)theta)

In the case that the operator $A$ prepares a state with a uniform distribution, the magnitudes of good outcomes are given by the function below, where the parameter `n` is the number of qubits, `L` is the number of good outcomes, and `j` is the number of iterations.

In [25]:
from math import sin, asin

def target_amplitude_uniform(n, l, j):
    theta = asin(sqrt(l/2**n))
    return sin((2*j+1)*theta)/sqrt(l)

Let's apply one iteration of the Grover iterate to our example state, where $n = 3$ and the outcome 3 is the good outcome:

In [26]:
n = 3
items = [3]
predicate = lambda i: True if i in items else False

state = [1/sqrt(2**n) for _ in range(2**n)]

classical_grover(state, predicate, iterations = 1)

assert is_close(state[items[0]], target_amplitude_uniform(3, 1, 1))

In [27]:
print_state_table(state)


Outcome  Binary  Amplitude           Magnitude  Direction  Amplitude Bar             Probability
------------------------------------------------------------------------------------------------
0        000     0.1768 + i0.0000    0.1768        0.00°   ████                      0.0312
1        001     0.1768 + i0.0000    0.1768        0.00°   ████                      0.0312
2        010     0.1768 + i0.0000    0.1768        0.00°   ████                      0.0312
3        011     0.8839 + i0.0000    0.8839        0.00°   █████████████████████     0.7812
4        100     0.1768 + i0.0000    0.1768        0.00°   ████                      0.0312
5        101     0.1768 + i0.0000    0.1768        0.00°   ████                      0.0312
6        110     0.1768 + i0.0000    0.1768        0.00°   ████                      0.0312
7        111     0.1768 + i0.0000    0.1768        0.00°   ████                      0.0312



Let's apply another iteration:

In [28]:
n = 3
items = [3]
predicate = lambda i: True if i in items else False

state = [1/sqrt(2**n) for _ in range(2**n)]

classical_grover(state, predicate, iterations = 2)

assert is_close(state[items[0]], target_amplitude_uniform(3, 1, 2))

In [29]:
print_state_table(state)


Outcome  Binary  Amplitude           Magnitude  Direction  Amplitude Bar             Probability
------------------------------------------------------------------------------------------------
0        000    -0.0884 + i0.0000    0.0884      180.00°   ██                        0.0078
1        001    -0.0884 + i0.0000    0.0884      180.00°   ██                        0.0078
2        010    -0.0884 + i0.0000    0.0884      180.00°   ██                        0.0078
3        011     0.9723 + i0.0000    0.9723        0.00°   ███████████████████████   0.9453
4        100    -0.0884 + i0.0000    0.0884      180.00°   ██                        0.0078
5        101    -0.0884 + i0.0000    0.0884      180.00°   ██                        0.0078
6        110    -0.0884 + i0.0000    0.0884      180.00°   ██                        0.0078
7        111    -0.0884 + i0.0000    0.0884      180.00°   ██                        0.0078



Let's try three iterations:

In [30]:
n = 3
items = [3]
predicate = lambda i: True if i in items else False

state = [1/sqrt(2**n) for _ in range(2**n)]

classical_grover(state, predicate, iterations = 3)

assert is_close(state[items[0]], target_amplitude_uniform(3, 1, 3))

In [31]:
print_state_table(state)


Outcome  Binary  Amplitude           Magnitude  Direction  Amplitude Bar             Probability
------------------------------------------------------------------------------------------------
0        000    -0.3094 + i0.0000    0.3094      180.00°   ███████                   0.0957
1        001    -0.3094 + i0.0000    0.3094      180.00°   ███████                   0.0957
2        010    -0.3094 + i0.0000    0.3094      180.00°   ███████                   0.0957
3        011     0.5745 + i0.0000    0.5745        0.00°   █████████████             0.3301
4        100    -0.3094 + i0.0000    0.3094      180.00°   ███████                   0.0957
5        101    -0.3094 + i0.0000    0.3094      180.00°   ███████                   0.0957
6        110    -0.3094 + i0.0000    0.3094      180.00°   ███████                   0.0957
7        111    -0.3094 + i0.0000    0.3094      180.00°   ███████                   0.0957



Using the number of good outcomes, `L = len(items)`, and the total number of outcomes, `2**n`, we can find the optimal number of iterations with:

In [32]:
from math import floor, pi

num_iterations = int(floor(pi/4*sqrt(2**n/len(items))))

Let's look at another example, this time using a random two-qubit state and good outcome 1:

In [33]:
n = 3
items = [0, 1]
predicate = lambda i: True if i in items else False
iterations = int(floor(pi/4*sqrt(2**n/len(items))))

state = generate_state(n)
classical_grover(state, predicate, iterations)

In [34]:
print_state_table(state)


Outcome  Binary  Amplitude           Magnitude  Direction  Amplitude Bar             Probability
------------------------------------------------------------------------------------------------
0        000    -0.4842 - i0.3511    0.5981     -144.95°   ██████████████            0.3577
1        001     0.5418 + i0.0665    0.5459        7.00°   █████████████             0.298 
2        010     0.1682 + i0.1715    0.2403       45.56°   █████                     0.0577
3        011     0.3003 + i0.0666    0.3076       12.50°   ███████                   0.0946
4        100    -0.1251 + i0.0109    0.1256      175.20°   ███                       0.0158
5        101     0.1986 + i0.0538    0.2058       15.16°   ████                      0.0424
6        110    -0.1056 - i0.2127    0.2375     -116.60°   █████                     0.0564
7        111     0.1968 + i0.1967    0.2782       44.99°   ██████                    0.0774



### A classical but quantum-friendly implementation of the inversion operator (section 6.1.5)

In this section, we will use a randomly generated operator $A$. To create such an operator for $n$ qubits, we will use the function `random_transformation` defined in util.py. 

Let's define a random transformation (and its inverse) for $n = 3$ qubits:

In [35]:
from util import random_transformation

n = 3
f = random_transformation(n)
A = f[0]
A_inverse = f[1]

Let's look at the state prepared by this random operator:

In [36]:
from sim_core import init_state

state = init_state(n)
A(state)

In [37]:
print_state_table(state)


Outcome  Binary  Amplitude           Magnitude  Direction  Amplitude Bar             Probability
------------------------------------------------------------------------------------------------
0        000    -0.4312 + i0.1982    0.4746      155.31°   ███████████               0.2252
1        001     0.1808 + i0.2393    0.2999       52.93°   ███████                   0.09  
2        010     0.2025 + i0.0852    0.2197       22.82°   █████                     0.0483
3        011     0.2435 - i0.3402    0.4184      -54.59°   ██████████                0.175 
4        100     0.4396 - i0.2543    0.5079      -30.95°   ████████████              0.258 
5        101     0.1175 + i0.1076    0.1593       42.48°   ███                       0.0254
6        110     0.2067 - i0.1749    0.2707      -40.76°   ██████                    0.0733
7        111     0.3042 - i0.1112    0.3239      -20.92°   ███████                   0.1049



Listing 6.4 Function to perform the inversion operation

In [38]:
from math import log2

def inversion_0_transformation(f, state):
    n = int(log2(len(state)))

    transform = f[0]
    inverse_transform = f[1]
    
    inverse_transform(state) # applies inverse of operator A
    assert is_close(state[0].imag, 0)
    for k in range(1, len(state)): # applies the operator M_0
        state[k] = -state[k]
    transform(state) # applies operator A

Let's use the same operator $A$ we used to create the state above and apply an oracle for the good outcome 3:

In [39]:
predicate = lambda k: True if k == 3 else False
oracle(state, predicate)

In [40]:
print_state_table(state)


Outcome  Binary  Amplitude           Magnitude  Direction  Amplitude Bar             Probability
------------------------------------------------------------------------------------------------
0        000    -0.4312 + i0.1982    0.4746      155.31°   ███████████               0.2252
1        001     0.1808 + i0.2393    0.2999       52.93°   ███████                   0.09  
2        010     0.2025 + i0.0852    0.2197       22.82°   █████                     0.0483
3        011    -0.2435 + i0.3402    0.4184      125.59°   ██████████                0.175 
4        100     0.4396 - i0.2543    0.5079      -30.95°   ████████████              0.258 
5        101     0.1175 + i0.1076    0.1593       42.48°   ███                       0.0254
6        110     0.2067 - i0.1749    0.2707      -40.76°   ██████                    0.0733
7        111     0.3042 - i0.1112    0.3239      -20.92°   ███████                   0.1049



Now, we can apply the inversion operation to the state:

In [41]:
inversion_0_transformation(f, state)

In [42]:
print_state_table(state)


Outcome  Binary  Amplitude           Magnitude  Direction  Amplitude Bar             Probability
------------------------------------------------------------------------------------------------
0        000    -0.1293 + i0.0595    0.1423      155.28°   ███                       0.0203
1        001     0.0542 + i0.0718    0.09         52.95°   ██                        0.0081
2        010     0.0607 + i0.0255    0.0659       22.79°   █                         0.0043
3        011     0.5601 - i0.7824    0.9622      -54.60°   ███████████████████████   0.9258
4        100     0.1319 - i0.0763    0.1523      -30.95°   ███                       0.0232
5        101     0.0352 + i0.0323    0.0478       42.54°   █                         0.0023
6        110     0.0620 - i0.0525    0.0812      -40.74°   █                         0.0066
7        111     0.0912 - i0.0334    0.0971      -20.89°   ██                        0.0094



### Quantum Oracle (section 6.2.1)

Listing 6.5 Function to create a phase oracle circuit

In [43]:
from sim_circuit import *

def is_bit_not_set(m, k):
    return not (m & (1 << k))

def phase_oracle_match(n, items):
    q = QuantumRegister(n)
    qc = QuantumCircuit(q)

    for m in items:
        for i in range(n):
            if is_bit_not_set(m, i):
                qc.x(q[i])

        qc.mcp(pi, [q[i] for i in range(len(q) - 1)], q[len(q) - 1])

        for i in range(n):
            if is_bit_not_set(m, i):
                qc.x(q[i])
    return qc

### The inversion operator (section 6.2.2)

Listing 6.6 Function to create a circuit for `n` qubits that multiplies outcome 0 by -1

In [44]:
def inversion_0_circuit(n):
    q = QuantumRegister(n)
    qc = QuantumCircuit(q)

    for i in range(n):
        qc.x(q[i])

    qc.mcp(pi, [q[i] for i in range(n - 1)], q[n - 1])

    for i in range(n):
        qc.x(q[i])

    return qc

Listing 6.7 Function to create the inversion circuit

In [45]:
def inversion_circuit(A):
    n = sum(A.regs)
    q = QuantumRegister(n)
    qc = QuantumCircuit(q)

    qc.append(A.inverse(), q)

    qc.append(inversion_0_circuit(n), q)

    qc.append(A, q)

    return qc

### Grover iterate (section 6.2.3)

Listing 6.8 Function to create the Grover iterate circuit

In [46]:
def grover_iterate_circuit(A, O):
    n = sum(O.regs)
    q = QuantumRegister(n)
    qc = QuantumCircuit(q)

    qc.append(O, q)

    qc.append(inversion_circuit(A), q)

    return qc

### Putting it all together: Grover's Algorithm (section 6.2.4)

Listing 6.9 Function to create the magnitude amplification circuit

In [47]:
def grover_circuit(A, O, iterations):
    n = sum(A.regs)
    q = QuantumRegister(n)
    qc = QuantumCircuit(q)
    
    qc.append(A, q)

    for i in range(1, iterations + 1):
        qc.append(grover_iterate_circuit(A, O), q)
        qc.report(f'iteration_{i}')

    return qc

We can use the function `uniform` from chapter 4 to create a circuit $A$ that prepares a state with equal magnitudes:

In [48]:
def uniform(n):
    q = QuantumRegister(n)
    qc = QuantumCircuit(q)

    for i in range(len(q)):
        qc.h(q[i])

    return qc

With `uniform` as our operator $A$, $n = 3$ qubits, and good outcomes 1, 3, and 7, we can define the following circuit:

In [49]:
n = 3
items = [1, 3, 7]
num_iterations = int(floor(pi/4*sqrt(2**n/len(items))))

qc = grover_circuit(uniform(n), phase_oracle_match(n, items), num_iterations)

In [50]:
from util_qiskit import print_circuit

print_circuit(qc)

     ┌───┐                                         ┌───┐┌───┐       ┌───┐┌───┐
q_0: ┤ H ├──────■────────────────■───────────■─────┤ H ├┤ X ├─■─────┤ X ├┤ H ├
     ├───┤┌───┐ │     ┌───┐      │           │     ├───┤├───┤ │     ├───┤├───┤
q_1: ┤ H ├┤ X ├─■─────┤ X ├──────■───────────■─────┤ H ├┤ X ├─■─────┤ X ├┤ H ├
     ├───┤├───┤ │P(π) ├───┤┌───┐ │P(π) ┌───┐ │P(π) ├───┤├───┤ │P(π) ├───┤├───┤
q_2: ┤ H ├┤ X ├─■─────┤ X ├┤ X ├─■─────┤ X ├─■─────┤ H ├┤ X ├─■─────┤ X ├┤ H ├
     └───┘└───┘       └───┘└───┘       └───┘       └───┘└───┘       └───┘└───┘


We can check the amplitudes at each step using the reports generated with the following code:

In [51]:
for i in range(1, num_iterations + 1):
    for m in items:
        assert is_close(qc.reports[f'iteration_{i}'][2][m], (-1)**i * target_amplitude_uniform(n, len(items), i))

In [52]:
print_state_table(qc.run())


Outcome  Binary  Amplitude           Magnitude  Direction  Amplitude Bar             Probability
------------------------------------------------------------------------------------------------
0        000     0.1768 + i0.0000    0.1768        0.00°   ████                      0.0312
1        001    -0.5303 + i0.0000    0.5303      180.00°   ████████████              0.2812
2        010     0.1768 + i0.0000    0.1768        0.00°   ████                      0.0312
3        011    -0.5303 + i0.0000    0.5303      180.00°   ████████████              0.2812
4        100     0.1768 + i0.0000    0.1768        0.00°   ████                      0.0312
5        101     0.1768 + i0.0000    0.1768        0.00°   ████                      0.0312
6        110     0.1768 + i0.0000    0.1768        0.00°   ████                      0.0312
7        111    -0.5303 + i0.0000    0.5303      180.00°   ████████████              0.2812

